# **Summarisation & Evasion Notebook (LB)**

# **1. Objective**

- Summarise banker answers into short, PRA relevant insights.
- Generate an evasion score to tag summaries.
- RAG pipeline to bring in relevant external documents (e.g. PRA risk definitions, regulatory news).
- Optional extension: validate flagged risks with external/regulatory news.

# **2. Set up Workspace**

In [3]:
# Import libraries
# Core python
import os
import numpy as np
import pandas as pd

# NLP & Summarisation
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
import spacy

# Evaluation
from rouge_score import rouge_scorer
import evaluate
from bert_score import score as bertscore 

# Retrieval
from sentence_transformers import SentenceTransformer 
import faiss
import chromadb
import langchain
import llama_index

# ML
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Visualisations
import matplotlib.pyplot as plt
import seaborn as sns 

# **3. Load the dataset**

In [14]:
# Load the dataset.
jpm_2025_df = pd.read_csv('../data/processed/jpm/all_jpm_2025.csv')

# View the data.
jpm_2025_df.head()

,question_number,answer_number,speaker_name,role,company,content,year,quarter,source_pdf
0,1,NaN,Ken Usdin,analyst,Autonomous Research,"Good morning, Jeremy. Wondering if you could s...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
1,1,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Sure, Ken. So I mean, at a high level, I would...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
2,2,NaN,Ken Usdin,analyst,Autonomous Research,Yeah. And just one question on the NII ex. Mar...,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
3,2,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Yeah, that's a good question, Ken. You're righ...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
4,2,2.0,Jamie Dimon,Chairman & Chief Executive Officer,JPMorganChase,In the curve basically.,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...


# **4. Summarisation**

# **5. Evasion Scoring**

# **6. RAG**